In [ ]:
import urllib
import requests
from pandas.io.json import json_normalize
import json
import pandas as pd
import datetime
import time

In [ ]:
url = 'https://rsbuddy.com/exchange/summary.json'
urljson = urllib.request.urlopen(url)
jsondata = json.loads(urljson.read().decode())

In [ ]:
#Get a list of all items
df = pd.read_json(path_or_buf='https://rsbuddy.com/exchange/summary.json',orient='index', convert_axes=True)
df = df[['id','name','buy_average','buy_quantity','sell_average','sell_quantity','overall_average','overall_quantity']]
data = df.sort_values(by=['id']).reset_index()
data = data.drop(labels='index',axis=1)

In [150]:
#Create test set of specified range number of items
startNum = 0
numItems = 3
tdata = data[startNum:numItems]
tdata

,id,name,buy_average,buy_quantity,sell_average,sell_quantity,overall_average,overall_quantity
0,2,Cannonball,219,947553,219,991512,219,1939065
1,6,Cannon base,202192,19,196844,11,200231,30
2,8,Cannon stand,189315,19,187334,12,188548,31


In [139]:
i = 0
urlquery = pd.DataFrame(columns=['id', 'name', 'data'])
#Iterate through all items, grab data from api and append to urlquery dataframe
while(i<tdata['id'].count()):
    key = tdata.iloc[i]['id']
    name = tdata.iloc[i]['name']
    print(name)
    #Attempt to get data from API, retry if HTTP error
    try:
        url = f'https://api.rsbuddy.com/grandExchange?a=graph&g=240&start=1474615279000&i={key}'
        tempDataFrame = pd.DataFrame()
        tempDataFrame = pd.read_json(path_or_buf=url,orient='records', convert_axes=False)
        urlquery = urlquery.append({'id':key, 'name': name, 'data':tempDataFrame}, ignore_index=True)
        i+=1
        print("It worked! On to the next one.")
    except:
        print("Got fucked. Trying again. :)")
        time.sleep(1)
        
#Reformat urlquery columns, convert timestamp from miliseconds to seconds, convert timestamp to date, and sort by date
for index, row in urlquery.iterrows():
    row['data'] = row['data'][['ts','buyingPrice','buyingCompleted','sellingPrice','sellingCompleted','overallPrice','overallCompleted']]
    row['data'] = row['data'].sort_values(by=['ts'],ascending=1).reset_index()
    count = 0 
    for ind, r in row['data'].iterrows():
        r['ts'] = r['ts']/1000
        row['data'].loc[ind, 'date'] = datetime.datetime.fromtimestamp(r['ts']).isoformat()
    row['data'] = row['data'].drop(labels='index',axis=1)

Empty DataFrame
Columns: [id, name, data]
Index: []
2
It worked! On to the next one.
6
Got fucked. Trying again. :)
6
Got fucked. Trying again. :)
6
Got fucked. Trying again. :)
6
It worked! On to the next one.
8
It worked! On to the next one.


In [147]:
urlquery.iloc[0]['data'][:100]

,buyingCompleted,buyingPrice,overallCompleted,overallPrice,sellingCompleted,sellingPrice,ts
0,2035229.0,224.0,3571637,223,1536408.0,223.0,1474603200000
1,1824092.0,229.0,2920855,228,1096763.0,228.0,1474617600000
2,2573115.0,230.0,4696411,229,2123296.0,229.0,1474632000000
3,3081875.0,226.0,5362912,225,2281037.0,224.0,1474646400000
4,2693098.0,228.0,4844232,227,2151134.0,225.0,1474660800000
5,2302011.0,227.0,4514199,226,2212188.0,225.0,1474675200000
6,1927665.0,225.0,3548452,224,1620787.0,224.0,1474689600000
7,2055325.0,227.0,3693494,227,1638169.0,226.0,1474704000000
8,2646175.0,225.0,4611767,224,1965592.0,224.0,1474718400000
9,2926014.0,222.0,5369812,221,2443798.0,220.0,1474732800000
